In [1]:
import numpy
import pandas as pd
import numpy as np
import nltk
from ast import literal_eval
import treetaggerwrapper as ttw
from nltk.stem.porter import *

C:\Users\Simone\anaconda3\lib\site-packages\treetaggerwrapper.py:739: FutureWarning: Possible nested set at position 8
  punct2find_re = re.compile("([^ ])([[" + ALONEMARKS + "])",
C:\Users\Simone\anaconda3\lib\site-packages\treetaggerwrapper.py:2043: FutureWarning: Possible nested set at position 152
  DnsHostMatch_re = re.compile("(" + DnsHost_expression + ")",
C:\Users\Simone\anaconda3\lib\site-packages\treetaggerwrapper.py:2067: FutureWarning: Possible nested set at position 409
  UrlMatch_re = re.compile(UrlMatch_expression, re.VERBOSE | re.IGNORECASE)
C:\Users\Simone\anaconda3\lib\site-packages\treetaggerwrapper.py:2079: FutureWarning: Possible nested set at position 192
  EmailMatch_re = re.compile(EmailMatch_expression, re.VERBOSE | re.IGNORECASE)


In [2]:
recipes = pd.read_csv("data/RAW_recipes.csv")
recipes["steps"] = recipes["steps"].apply(literal_eval)
recipes["ingredients"] = recipes["ingredients"].apply(literal_eval)
recipes["tags"] = recipes["tags"].apply(literal_eval)

In [10]:
recipes["name"][1]

'a bit different  breakfast pizza'

In [4]:
recipes["steps"][0]

['make a choice and proceed with recipe',
 'depending on size of squash , cut into half or fourths',
 'remove seeds',
 'for spicy squash , drizzle olive oil or melted butter over each cut squash piece',
 'season with mexican seasoning mix ii',
 'for sweet squash , drizzle melted honey , butter , grated piloncillo over each cut squash piece',
 'season with sweet mexican spice mix',
 'bake at 350 degrees , again depending on size , for 40 minutes up to an hour , until a fork can easily pierce the skin',
 'be careful not to burn the squash especially if you opt to use sugar or butter',
 'if you feel more comfortable , cover the squash with aluminum foil the first half hour , give or take , of baking',
 'if desired , season with salt']

In [5]:
recipes["ingredients"][0]

['winter squash',
 'mexican seasoning',
 'mixed spice',
 'honey',
 'butter',
 'olive oil',
 'salt']

In [4]:
preprocessed_sentence = ''
lemma_sentence = ''
def tree_tag(text, tagger):
    tagged_text = tagger.tag_text(text)
    return ttw.make_tags(tagged_text)

In [5]:
def tag_ingredients(ingredients):
    
    tagged_ingredients = []
    for ingredient in ingredients:
    
        sentence = tree_tag(ingredient, tagger)

        for tag in sentence:
        # remove special characters
            if isinstance(tag, ttw.Tag):

                # stem
                stem = stemmer.stem(tag.lemma)
                
                if tag.pos in ['NN0', 'NN1', 'NN2']:
                    tagged_ingredients.append(stem.split('|')[0])
    return tagged_ingredients


In [6]:
def tag_sentence(original_sentence, ingredients):
    
    sentence = tree_tag(original_sentence, tagger)

    step_ingredients = []
    step_activities = []
    
    for tag in sentence:
    # remove special characters
        if isinstance(tag, ttw.Tag):
            
            # stem
            stem = stemmer.stem(tag.lemma)
            if stem in ingredients:
                step_ingredients.append(tag.lemma.split('|')[0])
            if tag.pos in ['VVB','VVZ','VVI',
                           'VVD','VVN', 'VVP','VVG']:#VVG not sure
                step_activities.append(tag.lemma.split('|')[0])
                
    if len(step_activities) == 0:
        # default activity if no verb is present
        step_activities.append('use')

    return step_ingredients, step_activities

            

In [45]:
selectByName = True

categories =['cheesecake']
name = "chicken nuggets"
max_recipes = 1000
tagger = ttw.TreeTagger(TAGLANG='en')
stemmer = PorterStemmer()

case_id_col = []
event_id_col = []
order_col = []
activity_col = []
ingredient_col = []

event_cnt = 0 
recipe_cnt = 0

for i in range(0,recipes.shape[0]):

    if (not selectByName and len(np.intersect1d(categories,recipes["tags"][i])) == len(categories)) or (selectByName and name.lower() in str(recipes["name"][i]).lower()):
        #print(str(recipes["name"][i]))
        recipe_cnt += 1
        if recipe_cnt < max_recipes and recipes["minutes"][i] < 1440:
            ingredients = tag_ingredients(recipes["ingredients"][i])

            minutesPerStep = recipes["minutes"][i]/len(recipes["steps"][i])
            time = 0

            for original_sentence in recipes["steps"][i]:
                step_ingredients, step_activities = tag_sentence(original_sentence,ingredients)

                # synchronous events
                for ingredient in step_ingredients:
                    for activity in step_activities:
                        case_id_col.append(i)
                        event_id_col.append(event_cnt)
                        event_cnt += 1
                        order_col.append(time)

                        activity_col.append(activity)
                        ingredient_col.append(ingredient)

                time += minutesPerStep
            

In [46]:
recipe_log = pd.DataFrame({'case_id': case_id_col,
                           'event_id': event_id_col,
                           'order': order_col,
                           'activity': activity_col,
                           'ingredient': ingredient_col})
recipe_log

,case_id,event_id,order,activity,ingredient
0,1104,0,0.000000,cut,chicken
1,1104,1,0.000000,begin,chicken
2,1104,2,0.000000,heating,chicken
3,1104,3,0.000000,cut,breast
4,1104,4,0.000000,begin,breast
...,...,...,...,...,...
1079,230412,1079,15.000000,use,horseradish
1080,230412,1080,16.666667,mix,cream
1081,230412,1081,16.666667,mix,horseradish
1082,230412,1082,16.666667,mix,pepper


In [47]:
if selectByName:
    recipe_log.to_csv('log_' + name + '_1000.csv',index = False)
else:
    recipe_log.to_csv('log_' + '_'.join(name) + '_1000.csv',index = False)

In [36]:
recipe_log['ingredient'].unique()

array(['vinegar', 'maple', 'syrup', 'sauce', 'sesame', 'oil',
       'gingerroot', 'cabbage', 'onion', 'carrot', 'apple', 'mix',
       'coleslaw', 'egg', 'yolk', 'sugar', 'butter', 'salt', 'cream',
       'milk', 'cayenne', 'pepper', 'white', 'mustard', 'flour',
       'paprika', 'mayonnaise', 'buttermilk', 'tarragon', 'water',
       'celery', 'pineapple', 'ketchup', 'poppy', 'seed', 'lemon',
       'juice', 'broccoli', 'almond', 'bell', 'chili', 'peanut', 'snow',
       'pea', 'radish', 'cucumber', 'rice', 'miso', 'ginger', 'grapeseed',
       'scallions', 'mandarin', 'orange', 'bok', 'choy', 'slice', 'snap',
       'mushroom', 'ramen', 'noodle', 'chicken', 'cilantro', 'pickle',
       'avocado', 'tabasco', 'lime', 'vegetable', 'curry', 'powder',
       'raisin', 'bacon', 'barbecue', 'capsicum', 'spring', 'sausage',
       'bun', 'beer', 'tortilla', 'wedge', 'strawberry', 'cranberry',
       'cheese', 'parsley', 'splenda', 'garlic', 'walnut', 'zest',
       'blueberry', 'clove', 'nu

In [37]:
recipe_log['case_id'].unique()

array([   240,    451,   1016,   1443,   1709,   4079,   4966,   5513,
         5835,   6368,   6595,   7455,   7476,   9130,   9138,   9215,
         9216,   9217,   9218,   9219,   9415,   9457,   9525,   9594,
        10453,  11386,  11583,  12493,  12511,  18335,  19215,  20246,
        21340,  21936,  22181,  22325,  22464,  22709,  25443,  25445,
        25654,  25937,  26495,  26558,  26845,  28987,  29339,  29960,
        30031,  31193,  32095,  32187,  32455,  32942,  32954,  33038,
        33217,  33218,  33245,  33519,  34056,  34123,  34313,  34501,
        36236,  36237,  36238,  36239,  36613,  36660,  36773,  38023,
        38039,  38076,  38488,  38611,  41836,  42679,  43810,  43811,
        44843,  47272,  48281,  48291,  48304,  48305,  48353,  48354,
        48925,  53113,  53217,  53341,  54313,  54750,  56683,  56684,
        56685,  56686,  56687,  56688,  56690,  56691,  56692,  56693,
        56694,  56695,  56696,  56698,  56701,  56702,  56703,  56704,
      